In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('data (1).csv')
df

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,2016-06-27,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by Franc

In [3]:
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']


In [4]:
train.shape

(3975, 27)

In [5]:
test.shape

(378, 27)

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
train.replace(to_replace='[^a-zA-Z]', value=' ', regex=True, inplace=True)
test.replace(to_replace='[^a-zA-Z]', value=' ', regex=True, inplace=True)

In [8]:
train_headlines = []
test_headlines = []

for row in range(0, train.shape[0]):
  train_headlines.append(' '.join(str(x) for x in train.iloc[row, 0:25]))

for row in range(0, test.shape[0]):
  test_headlines.append(' '.join(str(x) for x in test.iloc[row, 0:25]))

In [9]:
train_headlines[0]

'           0 A  hindrance to operations   extracts from the leaked reports Scorecard Hughes  instant hit buoys Blues Jack gets his skates on at ice cold Alex Chaos as Maracana builds up for United Depleted Leicester prevail as Elliott spoils Everton s party Hungry Spurs sense rich pickings Gunners so wide of an easy target Derby raise a glass to Strupar s debut double Southgate strikes  Leeds pay the penalty Hammers hand Robson a youthful lesson Saints party like it s      Wear wolves have turned into lambs Stump mike catches testy Gough s taunt Langer escapes to hit     Flintoff injury piles on woe for England Hunters threaten Jospin with new battle of the Somme Kohl s successor drawn into scandal The difference between men and women Sara Denver  nurse turned solicitor Diana s landmine crusade put Tories in a panic Yeltsin s resignation caught opposition flat footed Russian roulette'

In [10]:
test_headlines[0]

'           1 Most cases of cancer are the result of sheer bad luck rather than unhealthy lifestyles  diet or even inherited genes  new research suggests  Random mutations that occur in DNA when cells divide are responsible for two thirds of adult cancers across a wide range of tissues  Iran dismissed United States efforts to fight Islamic State as a ploy to advance U S  policies in the region   The reality is that the United States is not acting to eliminate Daesh  They are not even interested in weakening Daesh  they are only interested in managing it  Poll  One in   Germans would join anti Muslim marches UK royal family s Prince Andrew named in US lawsuit over underage sex allegations Some    asylum seekers refused to leave the bus when they arrived at their destination in rural northern Sweden  demanding that they be taken back to Malm or  some big city   Pakistani boat blows self up after India navy chase  All four people on board the vessel from near the Pakistani port city of Ka

In [11]:
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Premalatha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from joblib import Parallel, delayed
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
def preprocess_headline(headline):
    # Tokenizing the news-title by words
    words = headline.split()
    
    # Removing the stopwords and stemming the words
    words = [ps.stem(word) for word in words if word.lower() not in stop_words]
    
    # Joining the stemmed words
    return ' '.join(words)
def preprocess_batch(headlines):
    return [preprocess_headline(headline) for headline in headlines]

# Define the number of CPU cores to use for parallel processing
num_cores = 4  # Adjust according to your system's capabilities

# Batch processing using parallel execution
train_corpus = Parallel(n_jobs=num_cores)(
    delayed(preprocess_batch)(train_headlines[i:i+100])  # Adjust batch size as needed
    for i in range(0, len(train_headlines), 100)  # Adjust batch size as needed
)

# Flatten the list of lists into a single list
train_corpus = [headline for sublist in train_corpus for headline in sublist]


In [13]:
def preprocess_headline(headline):
    words = headline.split()
    words = [ps.stem(word) for word in words if word.lower() not in stop_words]
    return ' '.join(words)

def preprocess_batch(headlines):
    return [preprocess_headline(headline) for headline in headlines]

# Batch processing using parallel execution
test_corpus = Parallel(n_jobs=num_cores)(
    delayed(preprocess_batch)(test_headlines[i:i+100])  # Adjust batch size as needed
    for i in range(0, len(test_headlines), 100)  # Adjust batch size as needed
)

# Flatten the list of lists into a single list
test_corpus = [headline for sublist in test_corpus for headline in sublist]


In [14]:
train_corpus[0:3]

['0 hindranc oper extract leak report scorecard hugh instant hit buoy blue jack get skate ice cold alex chao maracana build unit deplet leicest prevail elliott spoil everton parti hungri spur sens rich pick gunner wide easi target derbi rais glass strupar debut doubl southgat strike leed pay penalti hammer hand robson youth lesson saint parti like wear wolv turn lamb stump mike catch testi gough taunt langer escap hit flintoff injuri pile woe england hunter threaten jospin new battl somm kohl successor drawn scandal differ men women sara denver nurs turn solicitor diana landmin crusad put tori panic yeltsin resign caught opposit flat foot russian roulett',
 '0 scorecard best lake scene leader german sleaz inquiri cheerio boyo main recommend cubi kill fee cubi kill fee cubi kill fee hopkin furiou foster lack hannib appetit cubi kill fee tale two tail say like like say elbow eye nippl task forc assess risk asteroid collis found last critic list time live dear doctor irish court halt ira 

In [15]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000, ngram_range=(2,2))
X_train = cv.fit_transform(train_corpus).toarray()

In [16]:
X_test = cv.transform(test_corpus).toarray()

In [17]:
y_train = train['Label']
train = train.iloc[:, 3:28]
y_test = test['Label']
test = test.iloc[:, 3:28]

In [18]:
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [19]:
lr.score(X_train,y_train)

0.9919496855345912

In [20]:
lr.score(X_test,y_test)

0.828042328042328

In [21]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

MultinomialNB()

In [22]:
nb_classifier.score(X_train,y_train)

0.9021383647798742

In [23]:
nb_classifier.score(X_test,y_test)

0.8068783068783069

In [32]:
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer

In [35]:

from keras.preprocessing.sequence import pad_sequences

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_corpus)

# Convert text data to sequences of word indices
X_train_seq = tokenizer.texts_to_sequences(train_corpus)
X_test_seq = tokenizer.texts_to_sequences(test_corpus)

# Pad sequences to ensure uniform length
max_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')


In [37]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100))
model.add(LSTM(units=100))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
